### Preprocessing of X variables

In [1]:
import pandas as pd
import numpy as np
import datetime

### Change the data unit from days to weeks

In [2]:
def xParameter(dataSetName, rawDatas, df):
    
    datas = rawDatas.drop(labels=range(0,2), axis=0)
    
    step = 7
    
    price_datas = datas.iloc[:,4]
    weekly_datas = [price_datas[i:i+step] for i in range(0, len(price_datas), step)]

    new_weekly_datas=[]
    for weekly_data in weekly_datas:
        new_weekly_datas.append(np.mean(weekly_data[~pd.isnull(weekly_data)]))
    
    
    df['%s' %dataSetName] = new_weekly_datas
    print("%s,Finished."%dataSetName)

In [3]:
def dateProcess(df):
    
    finish_datas = pd.DataFrame()
    
    dates = df.loc[2:,"Date"]
    dates = dates.reset_index(drop=True)
    
    step=7
    weekly_dates = [dates[i] for i in range(0, len(dates), step)]
    
    finish_datas = pd.DataFrame({
        
        "Date":weekly_dates
    })
    
    return finish_datas

### Combine data into a DataFrame

In [15]:
def dataMerge(DataSetName):
    
    newestData = pd.read_csv("/Users/alston/1.Paper/1231Data/%s.csv" %DataSetName)
    newData = pd.read_excel("/Users/alston/1.Paper/2015_Data/%s.xls" %DataSetName)
    oldData = pd.read_csv("%s.csv" %DataSetName)
    
    processedData = pd.concat([oldData, newData,newestData])
    processedData = processedData.reset_index(drop=True)
    
    return processedData

### Execute pre-processing

In [16]:
inDataSets = ['CDCS' ,'ADCS', 'CDWS', 'GL', 'NDCS', 'NRBCON', 'SL', 'USKCSPT','ZSCS']

mergedData = dataMerge('CDCS')
finish_datas = dateProcess(mergedData)

for inDataSet in inDataSets:
    mergedData = dataMerge(inDataSet)
    xParameter(inDataSet, mergedData, finish_datas)

CDCS,Finished.
ADCS,Finished.
CDWS,Finished.
GL,Finished.
NDCS,Finished.
NRBCON,Finished.
SL,Finished.
USKCSPT,Finished.
ZSCS,Finished.


### Let the data start from 20111031

In [17]:
finish_datas = finish_datas[finish_datas["Date"] >= 20111031]
finish_datas.reset_index(drop=True)

,Date,CDCS,ADCS,CDWS,GL,NDCS,NRBCON,SL,USKCSPT,ZSCS
0,20111031,7864.00,2126.400000,423475.000000,1737.4640,18794.800000,2.63240,34.0906,NaN,1941.700000
1,20111107,7654.70,2116.470000,411505.000000,1779.5400,18253.000000,2.66380,34.5126,NaN,1932.150000
2,20111114,7625.90,2108.350000,401500.000000,1753.6120,17876.400000,2.54660,33.2776,NaN,1931.600000
3,20111121,7248.95,2017.100000,396255.000000,1689.1100,17287.800000,2.50780,31.7890,NaN,1910.550000
4,20111128,7684.15,2073.600000,388955.000000,1733.1720,17230.050000,2.55980,32.3996,NaN,2018.050000
...,...,...,...,...,...,...,...,...,...,...
474,20201130,7668.60,2036.040000,149680.000000,1819.9782,16083.100000,1.24800,23.7882,113.110000,2743.840000
475,20201207,7739.04,2008.474000,148695.000000,1849.2032,16782.612000,1.28260,24.1696,110.946000,2791.670000
476,20201214,7834.45,2030.134000,134185.000000,1861.7416,17461.050000,1.35660,25.0754,117.622500,2816.590000
477,20201221,7783.67,2007.680000,118830.000000,1873.4530,16885.700000,1.36525,25.6832,116.773333,2795.850000


### Preprocessing of Y variables

In [18]:
def dataMerge(DataSetName):
    newestData = pd.read_excel("/Users/alston/1.Paper/1231Data/%s.xls" %DataSetName)
    newData = pd.read_excel("/Users/alston/1.Paper/2015_Data/%s.xls" %DataSetName)
    
    processedData = pd.concat([newData,newestData])
    processedData = processedData.reset_index(drop=True)
    
    return processedData

In [19]:
import datetime
def get_week_monday(date):
    
    week_start_time = date - datetime.timedelta(days=date.weekday())
    return week_start_time

In [20]:
df = dataMerge('YangtzeCopper')
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
df

,Date,Maximum,Minimum,Average,Volatility
0,2011-11-01,59750,59650,59700,200
1,2011-11-02,58250,58100,58175,-1525
2,2011-11-03,58250,58150,58200,25
3,2011-11-04,59100,58950,59025,825
4,2011-11-07,58900,58800,58850,-175
...,...,...,...,...,...
2227,2020-12-25,58380,58360,58370,180
2228,2020-12-28,58870,58850,58860,490
2229,2020-12-29,58620,58600,58610,-250
2230,2020-12-30,58200,58180,58190,-420


In [21]:
i = 0

weeklyPrice = []
date =[]
count = 0


while (i < (len(df)-1)):
    
    date.append(get_week_monday(df['Date'][i]))
    avg = [df.loc[i,"Average"]]
    
    
    try:
        while (get_week_monday(df['Date'][i+1]) == get_week_monday(df['Date'][i])):
            avg.append(df.loc[i+1,"Average"])
            i+=1

    
    except KeyError:
        pass
    
    
    weeklyPrice.append(np.mean(avg))
    i+=1

In [22]:
yData = pd.DataFrame({
    
    "Date":date,
    "CCSP":weeklyPrice
})

yData

,Date,CCSP
0,2011-10-31,58775.0
1,2011-11-07,49595.0
2,2011-11-14,57775.0
3,2011-11-21,56015.0
4,2011-11-28,57365.0
...,...,...
465,2020-11-30,57448.0
466,2020-12-07,57418.0
467,2020-12-14,58154.0
468,2020-12-21,58320.0


In [23]:
finish_datas['Date'] = pd.to_datetime(finish_datas['Date'], format='%Y%m%d')
finalData = pd.merge(finish_datas,yData, on='Date', how='left')

In [24]:
sum(finalData["Date"]>="2011-10-31")

479

### Processed data before processing of rate & inflation rate

In [25]:
# pd.set_option('display.max_columns', 500)
# pd.options.display.max_columns = 200

pd.options.display.max_rows = 200

# len(finalData[finalData.isnull().values==True])
# finalData
finalData = finalData[finalData["Date"]>="2011-10-31"]
finalData = finalData.reset_index(drop=True)
print(len(finalData))
print(len(finalData[finalData["CCSP"].isnull().values==True]))

479
9


### Preprocessing of  Rate

In [26]:
def readFile(fileName):
    data = pd.read_csv("/Users/alston/1.Paper/Rate/%s.csv" %fileName)
    return data

In [27]:
def weeklyRateData(data):
    
    data['Date'] = pd.to_datetime(data['Date'], format='%Y/%m/%d')
    data = data.sort_values(by = 'Date')
    data = data.loc[:,['Date','Close']]
    data = data.reset_index(drop=True)
    
    i = 0
    weeklyRate = []
    date =[]
    count = 0

    while (i < (len(data)-1)):

        date.append(get_week_monday(data['Date'][i]))
        avg = [data.loc[i,"Close"]]

        try:
            while (get_week_monday(data['Date'][i+1]) == get_week_monday(data['Date'][i])):

                avg.append(data.loc[i+1,"Close"])
                i+=1

        except KeyError:
            pass


        weeklyRate.append(np.mean(avg))
        avg=[]
        i+=1
        
    return [date,weeklyRate]

In [28]:
rate_names = ["EURUSD","USDCLP","USDCNY", "USDPEN"]

rateFinalData = pd.DataFrame(columns=["Date"])


data = readFile("USDCLP")
rateFinalData["Date"] = weeklyRateData(data)[0]

for rate_name in rate_names:
    
    try:
        data = readFile(rate_name)
        data = pd.DataFrame({"Date":weeklyRateData(data)[0],
                                "%s"%rate_name:weeklyRateData(data)[1]})
    
        rateFinalData = pd.merge(rateFinalData, data)
    except ValueError:
        pass

In [29]:
finalData = pd.merge(finalData,rateFinalData)

### Preprocessing of Inflation rate

In [30]:
finalData["Date"][0].year 

2011

In [31]:
def readInflationFile(fileName):
    data = pd.read_csv("/Users/alston/1.Paper/Inflation/%s.csv" %fileName)
    return data

In [32]:
fileNames = ["USInflation","ChinaInflation"]
US = readInflationFile(fileNames[0])
China = readInflationFile(fileNames[1])

In [33]:
def addUSInflationData(date):
    
    for i in range(len(US)):
        if (date.year == US.iloc[i,0] and date.month == US.iloc[i,1]):
            return (US.iloc[i,2])

In [34]:
def addChinaInflationData(date):
    
    for i in range(len(China)):
        if (date.year == China.iloc[i,0] and date.month == China.iloc[i,1]):
            return (China.iloc[i,2])

In [35]:
finalData["USInflation"] = np.nan
finalData["ChinaInflation"] = np.nan

In [36]:
finalData['USInflation']=finalData['Date'].map(addUSInflationData)
finalData['ChinaInflation']=finalData['Date'].map(addChinaInflationData)
# df['USInflation'] = df['Date'].map(addInflationData)

In [37]:
finalData

,Date,CDCS,ADCS,CDWS,GL,NDCS,NRBCON,SL,USKCSPT,ZSCS,CCSP,EURUSD,USDCLP,USDCNY,USDPEN,USInflation,ChinaInflation
0,2011-10-31,7864.00,2126.400000,423475.000000,1737.4640,18794.800000,2.63240,34.0906,NaN,1941.700000,58775.0,1.37605,495.425,6.351125,2.707750,3.5,5.5
1,2011-11-07,7654.70,2116.470000,411505.000000,1779.5400,18253.000000,2.66380,34.5126,NaN,1932.150000,49595.0,1.36998,499.350,6.345040,2.705000,3.4,4.2
2,2011-11-14,7625.90,2108.350000,401500.000000,1753.6120,17876.400000,2.54660,33.2776,NaN,1931.600000,57775.0,1.35218,508.300,6.347400,2.702200,3.4,4.2
3,2011-11-21,7248.95,2017.100000,396255.000000,1689.1100,17287.800000,2.50780,31.7890,NaN,1910.550000,56015.0,1.33828,521.860,6.357800,2.703500,3.4,4.2
4,2011-11-28,7684.15,2073.600000,388955.000000,1733.1720,17230.050000,2.55980,32.3996,NaN,2018.050000,57365.0,1.33856,518.340,6.351040,2.701600,3.4,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,2020-11-30,7668.60,2036.040000,149680.000000,1819.9782,16083.100000,1.24800,23.7882,113.110000,2743.840000,57448.0,1.20752,753.994,6.556120,3.598740,1.2,-0.5
475,2020-12-07,7739.04,2008.474000,148695.000000,1849.2032,16782.612000,1.28260,24.1696,110.946000,2791.670000,57418.0,1.21072,740.076,6.537440,3.595660,1.4,0.2
476,2020-12-14,7834.45,2030.134000,134185.000000,1861.7416,17461.050000,1.35660,25.0754,117.622500,2816.590000,58154.0,1.22024,730.262,6.536920,3.587940,1.4,0.2
477,2020-12-21,7783.67,2007.680000,118830.000000,1873.4530,16885.700000,1.36525,25.6832,116.773333,2795.850000,58320.0,1.21962,717.992,6.539920,3.606575,1.4,0.2


### Dealing with missing values

In [41]:
finalData["CCSP"].interpolate(inplace=True)
finalData.drop(["USKCSPT"], axis=1)

,Date,CDCS,ADCS,CDWS,GL,NDCS,NRBCON,SL,ZSCS,CCSP,EURUSD,USDCLP,USDCNY,USDPEN,USInflation,ChinaInflation
0,2011-10-31,7864.00,2126.400000,423475.000000,1737.4640,18794.800000,2.63240,34.0906,1941.700000,58775.0,1.37605,495.425,6.351125,2.707750,3.5,5.5
1,2011-11-07,7654.70,2116.470000,411505.000000,1779.5400,18253.000000,2.66380,34.5126,1932.150000,49595.0,1.36998,499.350,6.345040,2.705000,3.4,4.2
2,2011-11-14,7625.90,2108.350000,401500.000000,1753.6120,17876.400000,2.54660,33.2776,1931.600000,57775.0,1.35218,508.300,6.347400,2.702200,3.4,4.2
3,2011-11-21,7248.95,2017.100000,396255.000000,1689.1100,17287.800000,2.50780,31.7890,1910.550000,56015.0,1.33828,521.860,6.357800,2.703500,3.4,4.2
4,2011-11-28,7684.15,2073.600000,388955.000000,1733.1720,17230.050000,2.55980,32.3996,2018.050000,57365.0,1.33856,518.340,6.351040,2.701600,3.4,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,2020-11-30,7668.60,2036.040000,149680.000000,1819.9782,16083.100000,1.24800,23.7882,2743.840000,57448.0,1.20752,753.994,6.556120,3.598740,1.2,-0.5
475,2020-12-07,7739.04,2008.474000,148695.000000,1849.2032,16782.612000,1.28260,24.1696,2791.670000,57418.0,1.21072,740.076,6.537440,3.595660,1.4,0.2
476,2020-12-14,7834.45,2030.134000,134185.000000,1861.7416,17461.050000,1.35660,25.0754,2816.590000,58154.0,1.22024,730.262,6.536920,3.587940,1.4,0.2
477,2020-12-21,7783.67,2007.680000,118830.000000,1873.4530,16885.700000,1.36525,25.6832,2795.850000,58320.0,1.21962,717.992,6.539920,3.606575,1.4,0.2


### Export the final data

In [42]:
path = "ProcessedData.csv"
finalData.to_csv(path, encoding="Big5", index=False)